In [23]:
import pandas as pd
import numpy as np
from cryodrgn.starfile import Starfile
from cryodrgn import analysis, utils, config

import pickle
import os, sys
import re
import argparse
import matplotlib.pyplot as plt
log = print
%matplotlib inline
from cryosparc.tools import CryoSPARC

In [24]:
license = "XXXX"
email = "XXXX"
password = "XXXX"

In [3]:
cs = CryoSPARC(
    license=license,
    email=email,
    password=password,
    host="XXXX",
    base_port=39000,
)

In [25]:
def per_conf_fsc_gen_vols(cs, project_num, job_num, num_imgs):
    project = cs.find_project(project_num)
    job = cs.find_job(project_num, job_num) # Flex train
    particles = job.load_output("particles")

    job = project.create_external_job("W1", "Custom Latents")
    job.connect("particles", job_num, "particles", slots=["components"])

    v = np.empty((len(particles),2))
    for i in range(2):
        v[:,i] = particles[f'components_mode_{i}/value']

#     latent_pts = v[::num_imgs]
    z_lst = []
    z_mean_lst = [] 
    for i in range(num_vols):
        z_nth = v[i*num_imgs:(i+1)*num_imgs]
        z_nth_avg = z_nth.mean(axis=0)
        z_nth_avg = z_nth_avg.reshape(1,-1)
        z_lst.append(z_nth)
        z_mean_lst.append(z_nth_avg)
    
    nearest_z_lst = []
    centers_ind_lst = []
    for i in range(num_vols):
        nearest_z, centers_ind = analysis.get_nearest_point(z_lst[i], z_mean_lst[i])
        nearest_z_lst.append(nearest_z.reshape(nearest_z.shape[-1]))
        centers_ind_lst.append(centers_ind)
    latent_pts = np.array(nearest_z_lst)
    
    latents_dset = job.add_output(
        type="particle",
        name="latents",
        slots=[{"prefix": "components_mode_%d" % k, "dtype": "components", "required": True} for k in range(2)],
        title="Latents",
        alloc=len(latent_pts),
    )

    for k in range(2):
        latents_dset["components_mode_%d/component" % k] = k
        latents_dset["components_mode_%d/value" % k] = latent_pts[:, k]

    with job.run():
        job.save_output("latents", latents_dset)

### Dihedrals

In [5]:
project_num = "P572"
job_num = "J78"
num_imgs = 1000

per_conf_fsc_gen_vols(cs, project_num, job_num, num_imgs)